# Model 2: ARIMA, SARIMAX 

In [1]:
# Import filter warnings

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
# Import merged_data file

import pandas as pd
merged_data = pd.read_csv("merged_data.csv")

In [3]:
merged_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,sell_price,date,d,event_name,event_type,sales,total_revenue
0,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,2014-02-01,1100,Ramadan starts,Religious,208.8,208.8
1,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,2014-02-02,1101,SuperBowl,Sporting,208.8,417.6
2,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,2014-02-03,1102,Ramadan starts,Religious,208.8,626.4
3,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,2014-02-04,1103,Ramadan starts,Religious,208.8,835.2
4,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,2014-02-05,1104,Ramadan starts,Religious,208.8,1044.0


In [4]:
# Convert date into the date-time index

merged_data['date'] = pd.to_datetime(merged_data['date'])
merged_data.set_index('date', inplace=True)

In [5]:
merged_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,sell_price,d,event_name,event_type,sales,total_revenue
date,,,,,,,,,,,,,
2014-02-01,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1100,Ramadan starts,Religious,208.8,208.8
2014-02-02,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1101,SuperBowl,Sporting,208.8,417.6
2014-02-03,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1102,Ramadan starts,Religious,208.8,626.4
2014-02-04,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1103,Ramadan starts,Religious,208.8,835.2
2014-02-05,FOODS_3_737_WI_3_evaluation,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1104,Ramadan starts,Religious,208.8,1044.0


# Assign X and Y

In [6]:
# Assigned X

X = merged_data.drop(labels=["id","sales"],axis=1)

In [7]:
# Assign Y target variable

y = merged_data[["total_revenue"]]

In [8]:
X.head()

,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,sell_price,d,event_name,event_type,total_revenue
date,,,,,,,,,,,
2014-02-01,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1100,Ramadan starts,Religious,208.8
2014-02-02,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1101,SuperBowl,Sporting,417.6
2014-02-03,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1102,Ramadan starts,Religious,626.4
2014-02-04,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1103,Ramadan starts,Religious,835.2
2014-02-05,FOODS_3_737,FOODS_3,FOODS,WI_3,WI,11401,3.48,1104,Ramadan starts,Religious,1044.0


In [9]:
y.head()

,total_revenue
date,
2014-02-01,208.8
2014-02-02,417.6
2014-02-03,626.4
2014-02-04,835.2
2014-02-05,1044.0


In [10]:
#Missing data treatment

X.isna().sum()

item_id          0
dept_id          0
cat_id           0
store_id         0
state_id         0
wm_yr_wk         0
sell_price       0
d                0
event_name       0
event_type       0
total_revenue    0
dtype: int64

# Feature Scaling

In [11]:
# Split data into cat and con

cat = []
con = []
for i in X.columns:
    if(X[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [12]:
cat

['item_id',
 'dept_id',
 'cat_id',
 'store_id',
 'state_id',
 'event_name',
 'event_type']

In [13]:
con

['wm_yr_wk', 'sell_price', 'd', 'total_revenue']

In [14]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the scaler on the continuous features
X[con] = scaler.fit_transform(X[con])


In [15]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical feature
for feature in cat:
    X[feature] = label_encoder.fit_transform(X[feature])


In [16]:
X

,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,sell_price,d,event_name,event_type,total_revenue
date,,,,,,,,,,,
2014-02-01,0,0,0,0,0,0.138815,0.746207,-0.013166,24,2,-1.024895
2014-02-02,0,0,0,0,0,0.138815,0.746207,-0.011329,26,3,-1.020861
2014-02-03,0,0,0,0,0,0.138815,0.746207,-0.009493,24,2,-1.016828
2014-02-04,0,0,0,0,0,0.138815,0.746207,-0.007656,24,2,-1.012794
2014-02-05,0,0,0,0,0,0.138815,0.746207,-0.005820,24,2,-1.008760
...,...,...,...,...,...,...,...,...,...,...,...
2014-01-27,88,0,0,0,0,-0.182928,-0.985406,-0.022349,24,2,-0.552506
2014-01-28,88,0,0,0,0,-0.182928,-0.985406,-0.020512,24,2,-0.658326
2014-01-29,88,0,0,0,0,-0.182928,-0.985406,-0.018676,24,2,-0.764147


# Remove Outliers

In [17]:
import numpy as np
from scipy import stats

# Calculate Z-scores for each column in your DataFrame 'X'
z_scores = np.abs(stats.zscore(X, axis=0))

z_score_threshold = 3  # You can adjust this threshold as needed

outliers_mask = z_scores > z_score_threshold

outliers = X[outliers_mask.any(axis=1)]

X_clean = X[~outliers_mask.any(axis=1)]

In [18]:
X_clean

,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,sell_price,d,event_name,event_type,total_revenue
date,,,,,,,,,,,
2014-02-01,0,0,0,0,0,0.138815,0.746207,-0.013166,24,2,-1.024895
2014-02-03,0,0,0,0,0,0.138815,0.746207,-0.009493,24,2,-1.016828
2014-02-04,0,0,0,0,0,0.138815,0.746207,-0.007656,24,2,-1.012794
2014-02-05,0,0,0,0,0,0.138815,0.746207,-0.005820,24,2,-1.008760
2014-02-06,0,0,0,0,0,0.138815,0.746207,-0.003983,24,2,-1.004726
...,...,...,...,...,...,...,...,...,...,...,...
2014-01-27,88,0,0,0,0,-0.182928,-0.985406,-0.022349,24,2,-0.552506
2014-01-28,88,0,0,0,0,-0.182928,-0.985406,-0.020512,24,2,-0.658326
2014-01-29,88,0,0,0,0,-0.182928,-0.985406,-0.018676,24,2,-0.764147


In [19]:
# List of outliers
outliers.count()

item_id          11349
dept_id          11349
cat_id           11349
store_id         11349
state_id         11349
wm_yr_wk         11349
sell_price       11349
d                11349
event_name       11349
event_type       11349
total_revenue    11349
dtype: int64

In [20]:
X_clean.shape

(131615, 11)

# ARIMA as SARIMAX

In [21]:
# Import libraries

import pandas as pd
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [22]:
# 'merged_data' with 'date' as the index and 'sales' as the column

train_size = int(len(merged_data) * 0.8)  # 80% for training, 20% for testing
train, test = merged_data[:train_size], merged_data[train_size:]

In [23]:
# Fit an auto ARIMA model to the training data

auto_arima_model = pm.auto_arima(train['sales'], seasonal=False, stepwise=True, suppress_warnings=True, error_action="ignore")
print(auto_arima_model.summary())


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:               114371
Model:               SARIMAX(0, 1, 0)   Log Likelihood            -1137004.285
Date:                Tue, 10 Oct 2023   AIC                        2274010.571
Time:                        12:20:51   BIC                        2274020.218
Sample:                             0   HQIC                       2274013.481
                             - 114371                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      2.527e+07   2.63e+04    961.385      0.000    2.52e+07    2.53e+07
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):           4382332.81
Prob(Q):                              1.00   Pr

In [24]:
# Specify order parameters

p, d, q = auto_arima_model.order

In [25]:
# Train the data and store result in sarimax_model_fit.

sarimax_model = SARIMAX(train['sales'], order=(p, d, q))
sarimax_model_fit = sarimax_model.fit(disp=False)

C:\ProgramData\Anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\ProgramData\Anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\ProgramData\Anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\ProgramData\Anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [26]:
# Prediction on specified 7 days

forecast_horizon = 7
forecast = sarimax_model_fit.get_forecast(steps=forecast_horizon)

C:\ProgramData\Anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [27]:
# Forecast evaluation

forecast_values = forecast.predicted_mean
confidence_intervals = forecast.conf_int()

In [28]:
# MSE (mean squared error)

mse = mean_squared_error(test['sales'][-7:], forecast_values)
print(f"Mean Squared Error (MSE): {mse}")

Mean Squared Error (MSE): 463632191.69441235


In [29]:
# RMSE (Root mean squared error)

import numpy as np
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 21532.120000000286
